In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
# Load morphometry data
df_all_cores = pd.read_csv("foram_frag_morphometry.csv", encoding='latin-1')
df_all_cores.head()

/var/folders/ml/v613w2ns1z31hpn87j8vcl8h0000gq/T/ipykernel_30038/2411630716.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_all_cores = pd.read_csv("foram_frag_morphometry.csv", encoding='latin-1')


,Sample,Species,Type,Diameter,Aream2,Areamm2,Perimeterm,Perimeterm2,ConvexAream2,convexAreamm2,...,Solidity,Roundnessm,Circularity,Mean,Stddev,StddevInvariant,Skewness,Kurtosis,5thmoment,6thmoment
0,BARP9403,Globigerina bulloides,Planktonic Formainifera,292.0,89868.5,0.066854,1189.727051,1.026140,92513.5,0.068821,...,0.971410,0.809051,0.797853,119.732765,17.067535,0.143685,-1.016231,4.345409,-10.348482,34.153194
1,BARP9403,Globigerina bulloides,Planktonic Formainifera,257.0,69480.0,0.051687,1095.401123,0.944784,72929.0,0.054252,...,0.952707,0.684475,0.727651,115.244179,16.724291,0.147724,-1.076683,3.959310,-9.469131,29.717978
2,BARP9403,Globigerina bulloides,Planktonic Formainifera,192.0,38891.0,0.028931,794.867065,0.685573,40312.0,0.029988,...,0.964750,0.924026,0.773517,102.506828,13.562996,0.132553,-1.175333,4.801376,-13.114857,46.299988
3,BARP9403,Globigerinella siphonifera,Planktonic Formainifera,231.0,56545.5,0.042065,1024.045776,0.883240,60057.5,0.044677,...,0.941523,0.723158,0.677593,101.876572,14.474765,0.141386,-0.046323,3.245234,0.432873,18.649073
4,BARP9403,Globigerinella siphonifera,Planktonic Formainifera,350.0,129298.5,0.096186,1513.633667,1.305509,136248.5,0.101356,...,0.948990,0.613363,0.709189,115.152351,15.260325,0.133828,-0.497275,4.133267,-6.165670,28.767853


In [20]:
# Separate foraminifera and fragment data
foram_data = df_all_cores[df_all_cores['Type'] == 'Planktonic Formainifera']
fragment_data = df_all_cores[df_all_cores['Type'] == 'Fragments']

In [21]:
# Drop rows with unknown Core ID
foram_data = foram_data[foram_data["Sample"] != "unknown"]
fragment_data = fragment_data[fragment_data["Sample"] != "unknown"]

In [22]:
# Confirm unique core ids 
foram_data['Sample'].unique()

array(['BARP9403', 'BARP9406', 'BARP9409', 'BARP9412', 'BARP9422',
       'BARP9426', 'BARP9430', 'BARP9434', 'BARP9437 ', 'BARP9439',
       'BARP9441', 'BARP9442', 'BARP9443', 'MDB04-2873',
       'MD04-2875B 0-2 Autolabel and Cleaned', 'MD04-2876',
       'MD04-2877 0-2 Autolable and Cleaned', 'MD12-3418C2', 'MD12-3423',
       'MD 76-011', 'MD76-132', 'MD76-133 ', 'MD76-136',
       'MD76-164 0-2 Autolabel and Cleaned', 'MD77-160 ', 'MD77-169',
       'MD77-171 ', 'MD77-178', 'MD77-180 ', 'MD77-182 ', 'MD77-184 ',
       'MD77-185 0-2 Autolabel and Cleaned', 'MD77-195', 'MD77-197',
       'MD77-200', 'MD77-202', 'MD77-204', 'MD77-205', 'MD79-256',
       'MD79-257', 'MD79-260d', 'MD79-261', 'MD79-275', 'MD79-276',
       'MD79-277 0-2 Autolable and Cleaned', 'MD81-345', 'MD85-640',
       'MD90-0936', 'MD90-0938', 'MD90-0939', 'MD90-0940',
       'MD90-0949 0-1 Autolabel and Cleaned',
       'MD90-O955 0-3 Autolabel and Cleaned',
       'MD90-0956 0-2 Autolabel and Cleaned', 'MD90-

In [23]:
# Strictly retrieve Core Ids only
foram_data['core_id'] = foram_data['Sample'].apply(lambda x: x.split(" ")[0])
fragment_data['core_id'] = fragment_data['Sample'].apply(lambda x: x.split(" ")[0])

In [24]:
# Replace wrong texts with their appropriate Core IDs in the Planktonic Formainifera data
foram_data['core_id'] = (foram_data['core_id'].replace("MD", "MD76-011")
                         .replace("MDB04-2873", "MD04-2873")
                         .replace("MD79-260d", "MD79-260")
                         .replace("BARDP9411", "BARP9411") 
                         .replace("MD90-O955", "MD90-0955") 
                         )


# Confirm string replacements
foram_data['core_id'].unique()

array(['BARP9403', 'BARP9406', 'BARP9409', 'BARP9412', 'BARP9422',
       'BARP9426', 'BARP9430', 'BARP9434', 'BARP9437', 'BARP9439',
       'BARP9441', 'BARP9442', 'BARP9443', 'MD04-2873', 'MD04-2875B',
       'MD04-2876', 'MD04-2877', 'MD12-3418C2', 'MD12-3423', 'MD76-011',
       'MD76-132', 'MD76-133', 'MD76-136', 'MD76-164', 'MD77-160',
       'MD77-169', 'MD77-171', 'MD77-178', 'MD77-180', 'MD77-182',
       'MD77-184', 'MD77-185', 'MD77-195', 'MD77-197', 'MD77-200',
       'MD77-202', 'MD77-204', 'MD77-205', 'MD79-256', 'MD79-257',
       'MD79-260', 'MD79-261', 'MD79-275', 'MD79-276', 'MD79-277',
       'MD81-345', 'MD85-640', 'MD90-0936', 'MD90-0938', 'MD90-0939',
       'MD90-0940', 'MD90-0949', 'MD90-0955', 'MD90-0956', 'MD90-0957',
       'MD90-0958', 'MD90-0959', 'MD90-0960', 'MD90-0961', 'MD90-0963',
       'MD96-2044', 'MD96-2045', 'MD96-2049', 'MD96-2051', 'MD96-2053',
       'MD96-2054', 'MD96-2055', 'MD96-2056', 'MD96-2058', 'MD96-2059',
       'MD96-2060', 'MD96-2061

In [25]:
# Replace wrong texts with their appropriate Core IDs in the fragments data
fragment_data['core_id'] = (fragment_data['core_id'].replace("MD", "MD76-011")
                         .replace("MDB04-2873", "MD04-2873")
                         .replace("MD79-260d", "MD79-260")
                         .replace("BARDP9411", "BARP9411") 
                         .replace("MD90-O955", "MD90-0955") 
                         )

# Confirm string replacements
fragment_data['core_id'].unique()

array(['BARP9411', 'BARP9403', 'BARP9406', 'BARP9409', 'BARP9412',
       'BARP9422', 'BARP9426', 'BARP9430', 'BARP9434', 'BARP9437',
       'BARP9439', 'BARP9441', 'BARP9442', 'BARP9443', 'MD04-2873',
       'MD04-2875B', 'MD04-2876', 'MD04-2877', 'MD12-3418C2', 'MD12-3423',
       'MD76-011', 'MD76-132', 'MD76-133', 'MD76-136', 'MD76-164',
       'MD77-160', 'MD77-169', 'MD77-171', 'MD77-178', 'MD77-180',
       'MD77-182', 'MD77-184', 'MD77-185', 'MD77-195', 'MD77-197',
       'MD77-200', 'MD77-202', 'MD77-204', 'MD77-205', 'MD79-256',
       'MD79-257', 'MD79-260', 'MD79-261', 'MD79-275', 'MD79-276',
       'MD79-277', 'MD81-345', 'MD85-640', 'MD90-0936', 'MD90-0938',
       'MD90-0939', 'MD90-0940', 'MD90-0949', 'MD90-0955', 'MD90-0956',
       'MD90-0957', 'MD90-0958', 'MD90-0959', 'MD90-0960', 'MD90-0961',
       'MD90-0963', 'MD96-2044', 'MD96-2045', 'MD96-2049', 'MD96-2051',
       'MD96-2053', 'MD96-2054', 'MD96-2055', 'MD96-2056', 'MD96-2058',
       'MD96-2059', 'MD96-2060'

In [26]:
# Load data containing target Core IDs
df_target_cores = pd.read_csv("dissolution_manuscript_final.csv")

In [27]:
# Extract target core ids
target_cores = df_target_cores.iloc[ : , 0]
target_cores = list(target_cores)

In [28]:
# Create new planktonic foraminifera DataFrame based on target Core IDs
df_foram = foram_data[foram_data['core_id'].isin(target_cores)]

In [29]:
# Create new fragment DataFrame based on target Core IDs
df_fragment = fragment_data[fragment_data['core_id'].isin(target_cores)]

In [30]:
# Confirm no target core is missing
missing_target_cores_foram = []

missing_target_cores_foram.extend(
    core for core in target_cores if core not in df_foram['core_id'].unique()
)

# Show list of missing target cores
print(missing_target_cores_foram) # Should return an empty list if no core is missing

[]


In [61]:
# Confirm no target core is missing
missing_target_cores_frag = []

missing_target_cores_frag.extend(
    core for core in target_cores if core not in df_fragment['core_id'].unique()
)

# Show list of missing target cores
print(missing_target_cores_frag) # Should return an empty list if no core is missing

[]


In [32]:
# Group each data by sample (Core ID)
forams_grouped = df_foram.groupby('core_id')
fragments_grouped = fragment_data.groupby('core_id')

## Remove Outliers

In [34]:
def remove_outliers_iqr(df, column):
    """Removes outliers from a DataFrame column using the IQR method.

    Args:
        df: The Pandas DataFrame.
        column: The name of the column to remove outliers from.

    Returns:
        A new DataFrame with the outliers removed.
    """

    # Calculate quantiles and IQR
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1

    # Calculate lower and upper bounds
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter data within bounds
    df_filtered = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

    return df_filtered

In [ ]:
def get_foram_working_data():
    """
    Processes foram data by removing outliers for each unique core_id and 
    concatenates the results into a single DataFrame.

    Returns:
        pd.DataFrame: A DataFrame containing the processed data for all samples.
    """
    result = pd.DataFrame()
    for sample in df_foram['core_id'].unique():
        foram_data = forams_grouped.get_group(sample)
        processed_data = remove_outliers_iqr(foram_data, 'Diameter')
        result = pd.concat([result, processed_data])
    return result

In [ ]:
def get_frag_working_data():
    """
    Processes fragment data by removing outliers for each unique core_id and 
    concatenates the results into a single DataFrame.

    Returns:
        pd.DataFrame: A DataFrame containing the processed data for all samples.
    """
    result = pd.DataFrame()
    for sample in df_fragment['core_id'].unique():
        fragment_data = fragments_grouped.get_group(sample)
        frag_processed_data = remove_outliers_iqr(fragment_data, 'Diameter')
        frag_result = pd.concat([frag_result, frag_processed_data])
    return frag_result

In [ ]:
# Store groups without outliers in a new variable
foram_df_no_outliers = get_foram_working_data()
frag_df_no_outliers = get_frag_working_data()

In [ ]:
# Create groups by unique core_id
foram_no_outlier_grouped = foram_df_no_outliers.groupby('core_id')
frag_no_outlier_grouped =  frag_df_no_outliers.groupby('core_id')

## Run Vif (multicollinearity test)

### Forams

In [43]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler

def calculate_vif(df, features):
    """Calculates VIF for specified features in a DataFrame.

    Args:
        df (pd.DataFrame): DataFrame containing the features.
        features (list): List of feature names for VIF calculation.

    Returns:
        pd.DataFrame: VIF results for each feature.
    """

    X = df[features]  # Select features for VIF
    X = X.dropna()  # Drop rows with missing values


    # Standardize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Create a DataFrame with the standardized data
    X_scaled_df = pd.DataFrame(X_scaled, columns=features)

    # Calculate VIF for each feature
    vif_data = pd.DataFrame()
    vif_data['Feature'] = X_scaled_df.columns
    vif_data['VIF'] = [variance_inflation_factor(X_scaled_df.values, i) for i in range(X_scaled_df.shape[1])]

    return vif_data

In [68]:
def create_vif_data():
    all_features = []  # List to store features for each sample

    for sample in foram_df_no_outliers['core_id'].unique():
        # Forams metrics
        foram_data = foram_no_outlier_grouped.get_group(sample)
        variance_forams = np.var(foram_data['Diameter'], ddof=1)  # Sample variance
        diameter_forams = foram_data['Diameter'].mean()
        circularity_forams = foram_data['Circularity'].mean()
        solidity_forams = foram_data['Solidity '].mean()
        eccentricity_forams = foram_data['Eccentricity'].mean()
        area_forams = foram_data['Areamm2'].mean()
        perimeter_forams = foram_data['Perimeterm2'].mean()
        roundness_forams = foram_data['Roundnessm'].mean()
        kurtosis_forams = foram_data['Kurtosis'].mean()

        # Append the feature list for the current sample
        all_features.append([variance_forams,  circularity_forams, 
                             
                             roundness_forams,   kurtosis_forams
        ])
    # diameter_forams, area_forams, eccentricity_forams,solidity_forams, perimeter_forams,
    # Create DataFrame after collecting all features
    vif_data = pd.DataFrame(all_features, columns=['variance_forams', 'circularity_forams',                            
                            'roundness_forams', 'kurtosis_forams'
    ])

    return vif_data

In [69]:
df_foram_X = create_vif_data()
feature_foram = ['variance_forams', 'circularity_forams', 
            
            'roundness_forams',  'kurtosis_forams'
        ]

calculate_vif(df_foram_X, feature_foram)

,Feature,VIF
0,variance_forams,1.505644
1,circularity_forams,1.974038
2,roundness_forams,1.899481
3,kurtosis_forams,1.904695


## Fragments

In [76]:
foram_no_outlier_grouped['core_id'].nunique()

core_id
BARP9409       1
BARP9411       1
BARP9412       1
BARP9422       1
BARP9426       1
              ..
MD96-2066      1
MD96-2067a     1
MD96-2067b     1
MD96-2067b2    1
MD98-2165      1
Name: core_id, Length: 62, dtype: int64

In [ ]:
df_fragment['core_id'].unique()

62

In [70]:
def create_frag_vif_data():
    all_features = []  # List to store features for each sample

    for sample in frag_df_no_outliers['core_id'].unique():
        # Forams metrics
        fragment_data = frag_no_outlier_grouped.get_group(sample)
        variance_frag = np.var(fragment_data['Diameter'], ddof=1)  # Sample variance
        diameter_frag = fragment_data['Diameter'].mean()
        circularity_frag = fragment_data['Circularity'].mean()
        solidity_frag = fragment_data['Solidity '].mean()
        eccentricity_frag = fragment_data['Eccentricity'].mean()
        area_frag = fragment_data['Areamm2'].mean()
        perimeter_frag = fragment_data['Perimeterm2'].mean()
        roundness_frag = fragment_data['Roundnessm'].mean()
        kurtosis_frag = fragment_data['Kurtosis'].mean()

        # Append the feature list for the current sample
        all_features.append([variance_frag, diameter_frag, circularity_frag,
              eccentricity_frag, kurtosis_frag, solidity_frag, area_frag,
              perimeter_frag, roundness_frag
        ])

    # Create DataFrame after collecting all features
    frag_vif_data = pd.DataFrame(all_features, columns=[
    'variance_frag', 'diameter_frag', 'circularity_frag',
              'eccentricity_frag', 'kurtosis_frag', 'solidity_frag', 'area_frag',
              'perimeter_frag', 'roundness_frag',
            
    ])

    return frag_vif_data

In [72]:
df_frag_X = create_frag_vif_data()
feature_frag = ['variance_frag', 'diameter_frag', 'circularity_frag',
              'eccentricity_frag', 'kurtosis_frag', 'solidity_frag', 'area_frag',
              'perimeter_frag', 'roundness_frag',]

calculate_vif(df_frag_X, feature_frag)

/Users/michaeladebayo/Documents/GitHub/code-for-manuscript-3/.venv/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


,Feature,VIF
0,variance_frag,NaN
1,diameter_frag,NaN
2,circularity_frag,NaN
3,eccentricity_frag,NaN
4,kurtosis_frag,NaN
5,solidity_frag,NaN
6,area_frag,NaN
7,perimeter_frag,NaN
8,roundness_frag,NaN


## Forams + Fragments

In [66]:
def create_full_vif_data():
    all_features = []  # List to store features for each core/sample

    # Get the common core IDs present in both datasets
    common_core_ids = set(df_foram['core_id'].unique()) & set(df_fragment['core_id'].unique())

    for core_id in common_core_ids:
        # Forams metrics
        foram_data = foram_no_outlier_grouped.get_group(core_id)
        variance_forams = np.var(foram_data['Diameter'], ddof=1)  # Sample variance
        diameter_forams = foram_data['Diameter'].mean()
        circularity_forams = foram_data['Circularity'].mean()
        solidity_forams = foram_data['Solidity '].mean()  # Removed extra space
        eccentricity_forams = foram_data['Eccentricity'].mean()
        area_forams = foram_data['Areamm2'].mean()
        perimeter_forams = foram_data['Perimeterm2'].mean()
        roundness_forams = foram_data['Roundnessm'].mean()
        kurtosis_forams = foram_data['Kurtosis'].mean()
        
        # Fragments metrics
        fragment_data = frag_no_outlier_grouped.get_group(core_id)
        variance_frag = np.var(fragment_data['Diameter'], ddof=1)
        diameter_frag = fragment_data['Diameter'].mean()
        circularity_frag = fragment_data['Circularity'].mean()
        solidity_frag = fragment_data['Solidity '].mean()
        eccentricity_frag = fragment_data['Eccentricity'].mean()
        area_frag = fragment_data['Areamm2'].mean()
        perimeter_frag = fragment_data['Perimeterm2'].mean()
        roundness_frag = fragment_data['Roundnessm'].mean()
        kurtosis_frag = fragment_data['Kurtosis'].mean()

        # Append combined features
        all_features.append([
            variance_forams, circularity_forams, eccentricity_forams,
             perimeter_forams, kurtosis_forams,
            variance_frag, circularity_frag, eccentricity_frag,
            perimeter_frag,  kurtosis_frag
        ])

    # Create DataFrame after collecting all features
    full_vif_data = pd.DataFrame(all_features, columns=[
        'variance_forams', 'circularity_forams',
        'eccentricity_forams', 'perimeter_forams',  'kurtosis_forams',
        'variance_frag', 'circularity_frag', 
        'eccentricity_frag',  'perimeter_frag',  'kurtosis_frag'
    ])
    
    return full_vif_data


In [67]:
df_full_X = create_full_vif_data()
feature = ['variance_forams', 'circularity_forams', 
        'eccentricity_forams',  'perimeter_forams', 'kurtosis_forams',
        'variance_frag',  'circularity_frag', 
        'eccentricity_frag', 'perimeter_frag',  'kurtosis_frag']

calculate_vif(df_full_X, feature)

KeyError: 'BARP9422'

## Formula 1 

In [ ]:
# def calculate_fv_index_1():
#     fv_index_results = []  # List to store FV-Index metrics for each sample

#     # Get the common core IDs present in both datasets
#     common_core_ids = set(df_foram['core_id'].unique()) & set(df_fragment['core_id'].unique())

#     for core_id in common_core_ids:

#         # Forams metrics
#         foram_data = forams_grouped.get_group(core_id)
#         variance_forams = np.var(foram_data['Diameter'], ddof=1)  # Sample variance
#         # diameter_forams = foram_data['Diameter'].mean()
#         circularity_forams = foram_data['Circularity'].mean()
#         # solidity_forams = foram_data['Solidity '].mean()  # Removed extra space
#         eccentricity_forams = foram_data['Eccentricity'].mean()
#         # area_forams = foram_data['Areamm2'].mean()
#         # perimeter_forams = foram_data['Perimeterm2'].mean()
#         # roundness_forams = foram_data['Roundnessm'].mean()
#         kurtosis_forams = foram_data['Kurtosis'].mean()
        
#         # Fragments metrics
#         fragment_data = fragments_grouped.get_group(core_id)
#         variance_frag = np.var(fragment_data['Diameter'], ddof=1)
#         # diameter_frag = fragment_data['Diameter'].mean()
#         # circularity_frag = fragment_data['Circularity'].mean()
#         # solidity_frag = fragment_data['Solidity '].mean()
#         eccentricity_frag = fragment_data['Eccentricity'].mean()
#         # area_frag = fragment_data['Areamm2'].mean()
#         # perimeter_frag = fragment_data['Perimeterm2'].mean()
#         # roundness_frag = fragment_data['Roundnessm'].mean()
#         # kurtosis_frag = fragment_data['Kurtosis'].mean()


#         # Compute FV-Index
#         if variance_forams > 0 and circularity_forams > 0:  # Avoid division by zero
#             fv_index = (variance_frag / variance_forams) * ((1-eccentricity_frag)/ circularity_forams) * (eccentricity_forams/kurtosis_forams)
#         else:
#             fv_index = np.nan  # Set to NaN if calculations are not valid

#         # Store the results
#         fv_index_results.append({
#             'Core ID': core_id,
#             'FV-Index': fv_index
#         })

#     FV_index_1 = pd.DataFrame(fv_index_results, columns=['Core ID', 'FV-Index'])

#     return FV_index_1


In [ ]:
# calculate_fv_index_1()

,Core ID,FV-Index
0,MD96-2058,0.064859
1,MD77-204,0.100022
2,MD90-0940,0.017526
3,MD90-0960,0.076423
4,MD96-2059,0.078336
...,...,...
57,MD90-0936,0.082904
58,BARP9422,0.044027
59,MD90-0963,0.049701
60,MD04-2875B,0.079848


## Formula 2

In [ ]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()

# def calculate_fv_index_2():
#     fv_index_results = []  # List to store FV-Index metrics for each sample

#     # Get the common core IDs present in both datasets
#     common_core_ids = set(df_foram['core_id'].unique()) & set(df_fragment['core_id'].unique())

#     for core_id in common_core_ids:

#         # Forams metrics
#         foram_data = forams_grouped.get_group(core_id)
#         variance_forams = np.var(foram_data['Diameter'], ddof=1)  # Sample variance
#         # diameter_forams = foram_data['Diameter'].mean()
#         circularity_forams = foram_data['Circularity'].mean()
#         # solidity_forams = foram_data['Solidity '].mean()  # Removed extra space
#         # eccentricity_forams = foram_data['Eccentricity'].mean()
#         # area_forams = foram_data['Areamm2'].mean()
#         # perimeter_forams = foram_data['Perimeterm2'].mean()
#         # roundness_forams = foram_data['Roundnessm'].mean()
#         kurtosis_forams = foram_data['Kurtosis'].mean()
        
#         # Fragments metrics
#         fragment_data = fragments_grouped.get_group(core_id)
#         variance_frag = np.var(fragment_data['Diameter'], ddof=1)
#         # diameter_frag = fragment_data['Diameter'].mean()
#         # circularity_frag = fragment_data['Circularity'].mean()
#         # solidity_frag = fragment_data['Solidity '].mean()
#         eccentricity_frag = fragment_data['Eccentricity'].mean()
#         area_frag = fragment_data['Areamm2'].mean()
#         perimeter_frag = fragment_data['Perimeterm2'].mean()
#         # roundness_frag = fragment_data['Roundnessm'].mean()
#         # kurtosis_frag = fragment_data['Kurtosis'].mean()


#         # Compute FV-Index
#         if variance_forams > 0 and circularity_forams > 0:  # Avoid division by zero
#             fv_index = ((variance_frag * (1 - eccentricity_frag)) / (variance_forams * (area_frag / perimeter_frag))) * kurtosis_forams
#         else:
#             fv_index = np.nan  # Set to NaN if calculations are not valid

#         # Store the results
#         fv_index_results.append({
#             'Core ID': core_id,
#             'FV-Index': fv_index
#         })

#     FV_index_2 = pd.DataFrame(fv_index_results, columns=['Core ID', 'FV-Index'])

#     FV_index_2['FV-Index'] = scaler.fit_transform(FV_index_2[['FV-Index']])

#     return FV_index_2


## Formula 3

In [ ]:
# def create_full_df():
#     all_features = []  # List to store features for each core/sample

#     # Get the common core IDs present in both datasets
#     common_core_ids = set(df_foram['core_id'].unique()) & set(df_fragment['core_id'].unique())

#     for core_id in common_core_ids:
#         # Forams metrics
#         foram_data = forams_grouped.get_group(core_id)
#         variance_forams = np.var(foram_data['Diameter'], ddof=1)  # Sample variance
#         diameter_forams = foram_data['Diameter'].mean()
#         circularity_forams = foram_data['Circularity'].mean()
#         solidity_forams = foram_data['Solidity '].mean()  # Removed extra space
#         eccentricity_forams = foram_data['Eccentricity'].mean()
#         area_forams = foram_data['Aream2'].mean()
#         perimeter_forams = foram_data['Perimeterm'].mean()
#         roundness_forams = foram_data['Roundnessm'].mean()
#         kurtosis_forams = foram_data['Kurtosis'].mean()
        
#         # Fragments metrics
#         fragment_data = fragments_grouped.get_group(core_id)
#         variance_frag = np.var(fragment_data['Diameter'], ddof=1)
#         diameter_frag = fragment_data['Diameter'].mean()
#         circularity_frag = fragment_data['Circularity'].mean()
#         solidity_frag = fragment_data['Solidity '].mean()
#         eccentricity_frag = fragment_data['Eccentricity'].mean()
#         area_frag = fragment_data['Aream2'].mean()
#         perimeter_frag = fragment_data['Perimeterm'].mean()
#         roundness_frag = fragment_data['Roundnessm'].mean()
#         kurtosis_frag = fragment_data['Kurtosis'].mean()

#         # Append combined features
#         all_features.append([
#             variance_forams, diameter_forams, circularity_forams, solidity_forams, eccentricity_forams,
#             area_forams, perimeter_forams, roundness_forams, kurtosis_forams, 
#             variance_frag, diameter_frag, circularity_frag, solidity_frag, eccentricity_frag,  area_frag,
#             perimeter_frag,  roundness_frag, kurtosis_frag
#         ])


#     # Create DataFrame after collecting all features
#     full_data_df = pd.DataFrame(all_features, columns=[
#             'variance_forams', 'diameter_forams', 'circularity_forams', 'solidity_forams', 'eccentricity_forams',
#             'area_forams', 'perimeter_forams', 'roundness_forams', 'kurtosis_forams', 
#             'variance_frag', 'diameter_frag', 'circularity_frag', 'solidity_frag', 'eccentricity_frag',  'area_frag',
#             'perimeter_frag',  'roundness_frag', 'kurtosis_frag'
#     ])

#     scaler = StandardScaler()
    
#     full_data_df = pd.DataFrame(scaler.fit_transform(full_data_df), columns=full_data_df.columns)
    
#     return full_data_df


In [ ]:
# # Load second dataset
# df_manu = pd.read_csv("dissolution_manuscript_final.csv")

# depth = df_manu['Depth (m)']
# omega_c = df_manu['Ω Calcite (CO3 2-)']

# env_param = pd.DataFrame({'Depth (m)': depth,
#                           'Ω Calcite (CO3 2-)': omega_c})

# env_param.head()

,Depth (m),Ω Calcite (CO3 2-)
0,3047,1.17
1,2198,1.25
2,3927,0.88
3,2060,1.31
4,2061,1.28


In [ ]:
# full_df = create_full_df()

In [ ]:
# merged_data = pd.concat([full_df, env_param], axis=1)

# merged_data.head()

,variance_forams,diameter_forams,circularity_forams,solidity_forams,eccentricity_forams,area_forams,perimeter_forams,roundness_forams,kurtosis_forams,variance_frag,diameter_frag,circularity_frag,solidity_frag,eccentricity_frag,area_frag,perimeter_frag,roundness_frag,kurtosis_frag,Depth (m),Ω Calcite (CO3 2-)
0,0.746944,0.831266,-1.118298,-0.789055,0.167861,0.767239,0.980847,-0.365904,-1.001124,1.399788,1.961048,-1.252650,-0.671532,1.273295,1.970700,2.129301,-1.142018,-0.313253,3047,1.17
1,-0.752422,-0.407796,0.065713,-0.859778,0.975902,-0.518150,-0.448997,-1.220131,-0.228918,-0.138779,0.164479,-0.971048,-1.093486,0.816559,0.075408,0.469582,-1.062118,0.525084,2198,1.25
2,4.746604,4.545724,1.806224,2.542232,-1.712366,5.234863,4.301101,1.281345,4.579109,2.666771,3.298124,-0.410955,-0.055600,1.136958,3.525407,3.190419,-1.115586,3.070741,3927,0.88
3,-0.791819,-0.670063,0.690446,-0.013314,0.603257,-0.700478,-0.777650,-0.624367,-0.636579,-0.872706,-0.322997,-0.591037,-0.809639,0.726012,-0.479967,-0.168558,-0.780683,0.648259,2060,1.31
4,-0.092665,-0.560597,-0.860273,-1.066913,-0.165610,-0.444466,-0.460940,-0.092135,-0.519967,0.387193,-0.494849,-0.549107,-0.327873,-0.328842,-0.331214,-0.397219,0.172071,-0.279049,2061,1.28


In [ ]:

# def calculate_fv_indices(df):
#     # Baseline Formula
#     df['FV_index_baseline'] = df['variance_frag'] / df['variance_forams']

#     # Option A: Adding Eccentricity
#     df['FV_index_eccentricity'] = (df['variance_frag'] / df['variance_forams']) * (1 - df['eccentricity_frag'])

#     # Option B: Using Area/Perimeter Ratio
#     df['FV_index_area_perimeter'] = (df['variance_frag'] / df['variance_forams']) * (df['area_frag'] / df['perimeter_frag'])

#     # Option C: Weighted Combination
#     alpha = 0.7  # Weight for variance
#     beta = 0.3   # Weight for eccentricity
#     df['FV_index_weighted'] = (alpha * (df['variance_frag'] / df['variance_forams'])) + (beta * (1 - df['eccentricity_frag']))

#     return df


In [ ]:
# Apply the function to dataset
# df = calculate_fv_indices(merged_data)

# df

,variance_forams,diameter_forams,circularity_forams,solidity_forams,eccentricity_forams,area_forams,perimeter_forams,roundness_forams,kurtosis_forams,variance_frag,...,area_frag,perimeter_frag,roundness_frag,kurtosis_frag,Depth (m),Ω Calcite (CO3 2-),FV_index_baseline,FV_index_eccentricity,FV_index_area_perimeter,FV_index_weighted
0,0.746944,0.831266,-1.118298,-0.789055,0.167861,0.767239,0.980847,-0.365904,-1.001124,1.399788,...,1.970700,2.129301,-1.142018,-0.313253,3047,1.17,1.874020,-0.512160,1.734434,1.229826
1,-0.752422,-0.407796,0.065713,-0.859778,0.975902,-0.518150,-0.448997,-1.220131,-0.228918,-0.138779,...,0.075408,0.469582,-1.062118,0.525084,2198,1.25,0.184443,0.033834,0.029619,0.184142
2,4.746604,4.545724,1.806224,2.542232,-1.712366,5.234863,4.301101,1.281345,4.579109,2.666771,...,3.525407,3.190419,-1.115586,3.070741,3927,0.88,0.561827,-0.076947,0.620818,0.352192
3,-0.791819,-0.670063,0.690446,-0.013314,0.603257,-0.700478,-0.777650,-0.624367,-0.636579,-0.872706,...,-0.479967,-0.168558,-0.780683,0.648259,2060,1.31,1.102153,0.301977,3.138371,0.853704
4,-0.092665,-0.560597,-0.860273,-1.066913,-0.165610,-0.444466,-0.460940,-0.092135,-0.519967,0.387193,...,-0.331214,-0.397219,0.172071,-0.279049,2061,1.28,-4.178425,-5.552468,-3.484103,-2.526245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,-0.764195,-0.365384,0.401438,0.158386,0.428692,-0.492840,-0.432783,-0.394198,-0.186449,-0.681736,...,-0.193476,-0.371845,0.280956,0.423566,1430,1.08,0.892097,0.967818,0.464169,0.949932
58,0.546708,1.310096,0.412662,0.478218,0.363244,1.089745,1.249050,-0.095650,1.075434,0.486248,...,1.181727,0.854016,-0.303900,-0.660330,2427,1.06,0.889411,0.073849,1.230705,0.647497
59,0.380094,-0.051162,-0.305836,-0.361108,0.531741,0.024359,-0.055471,-0.549382,1.834377,0.747158,...,0.474216,0.475218,-0.780271,2.152565,2200,1.13,1.965718,0.431185,1.961577,1.441809
60,-1.070061,-1.258522,-0.638348,-1.150257,0.897159,-1.140237,-1.227418,-1.267163,1.462189,-1.453137,...,-1.955849,-2.326481,2.150857,-1.130285,1854,1.12,1.357995,4.846447,1.141652,2.021244


In [ ]:
# # Apply the function to dataset
# dfc = calculate_fv_indices(merged_data)

# # Ensure the columns are present in the dataframe
# required_columns = ['FV_index_baseline', 'FV_index_eccentricity', 'FV_index_area_perimeter', 'FV_index_weighted']
# missing_columns = [col for col in required_columns if col not in dfc.columns]

# if missing_columns:
#     raise KeyError(f"Missing columns in dataframe: {missing_columns}")

# # Correlation with depth and Omega-calcite
# correlation_results = {
#     'Baseline': dfc[['FV_index_baseline', 'Depth (m)', 'Ω Calcite (CO3 2-)']].corr(),
#     'Eccentricity': dfc[['FV_index_eccentricity', 'Depth (m)','Ω Calcite (CO3 2-)']].corr(),
#     'Area_Perimeter': dfc[['FV_index_area_perimeter', 'Depth (m)','Ω Calcite (CO3 2-)']].corr(),
#     'Weighted': dfc[['FV_index_weighted', 'Depth (m)', 'Ω Calcite (CO3 2-)']].corr()
# }

# # Display correlation results
# for key, value in correlation_results.items():
#     print(f"\nCorrelation for {key} Formula:")
#     print(value[['Depth (m)', 'Ω Calcite (CO3 2-)']])


Correlation for Baseline Formula:
                    Depth (m)  Ω Calcite (CO3 2-)
FV_index_baseline   -0.096389            0.084993
Depth (m)            1.000000           -0.830126
Ω Calcite (CO3 2-)  -0.830126            1.000000

Correlation for Eccentricity Formula:
                       Depth (m)  Ω Calcite (CO3 2-)
FV_index_eccentricity  -0.173741            0.143891
Depth (m)               1.000000           -0.830126
Ω Calcite (CO3 2-)     -0.830126            1.000000

Correlation for Area_Perimeter Formula:
                         Depth (m)  Ω Calcite (CO3 2-)
FV_index_area_perimeter  -0.229480            0.160546
Depth (m)                 1.000000           -0.830126
Ω Calcite (CO3 2-)       -0.830126            1.000000

Correlation for Weighted Formula:
                    Depth (m)  Ω Calcite (CO3 2-)
FV_index_weighted   -0.140921            0.114541
Depth (m)            1.000000           -0.830126
Ω Calcite (CO3 2-)  -0.830126            1.000000


In [ ]:
# calculate_fv_index_2()

,Core ID,FV-Index
0,MD96-2058,-0.928166
1,MD77-204,1.046935
2,MD90-0940,-1.386945
3,MD90-0960,0.111414
4,MD96-2059,0.078421
...,...,...
57,MD90-0936,0.446778
58,BARP9422,-0.766334
59,MD90-0963,-0.086384
60,MD04-2875B,2.049570


In [ ]:
# # Load second dataset
# df_manu = pd.read_csv("dissolution_manuscript_final.csv")

# df_manu.head()

In [ ]:
# fv_data_2 = calculate_fv_index_2()

In [ ]:
# final_fv = df_manu.merge(fv_data_2, how='left', on='Core ID')

In [ ]:

# final_fv.to_csv("fvi_2.csv", index=False)
# # df_manu[df_manu['FV-Index', 'FV-Index']]